# Training for Applying Transformers and Explainable AI to Identify Partisan Keywords

see https://colab.research.google.com/drive/1mOSWKcD7pKSFSdyq2FBWTrhZ06SLTnLC?usp=sharing

In [ ]:
import numpy as np
import ktrain
from ktrain import text

In [1]:
import pandas as pd

pd.set_option('display.max_columns', 500)

df = pd.read_csv('~/data/domain_stop_removed_bias_text.csv')

df.head()

text_label = df[['text_stop_removed', 'audience_combined_bias']]

def replace_str(x):
    if x == 'left':
        return 0
    elif x == 'center':
        return 1
    else:
        return 2

text_label['label'] = text_label['audience_combined_bias'].apply(lambda x: replace_str(x))

text_label = text_label[['text_stop_removed', 'audience_combined_bias']]

text_label.columns = ['text', 'label']

text_label['text'] = text_label['text'].astype('string')

<ipython-input-1-85d20dfd0e52>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_label['label'] = text_label['audience_combined_bias'].apply(lambda x: replace_str(x))


In [ ]:
train_sample = text_label.groupby('label').sample(20000)

In [4]:
from sklearn.model_selection import train_test_split

In [ ]:
del df

In [ ]:
del text_label

In [5]:
#text_label = df[['text', 'label']]
x = text_label['text']
y = text_label['label']
train_x, test_x, train_y, test_y = train_test_split(np.array(x), np.array(y), test_size=.2, stratify=y)

In [ ]:
# x = np.array(x)
# y = np.array(y)
# (train_x, train_y), (test_x, test_y), preproc  = text.texts_from_array(x_train=x, y_train=y, class_names=['left', 'center', 'right'], preprocess_mode='distilbert')

In [10]:
MODEL_NAME = 'bert-base-uncased'

In [11]:
t = text.Transformer(MODEL_NAME, maxlen=250, classes=['left', 'center', 'right'])

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

In [12]:
train = t.preprocess_train(list(train_x), list(train_y))

preprocessing train...
language: en
train sequence lengths:
	mean : 313
	95percentile : 737
	99percentile : 1593


/opt/conda/envs/tranformer_tf_env/lib/python3.9/site-packages/ktrain/utils.py:589: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


Is Multi-Label? False


In [13]:
test = t.preprocess_test(list(test_x), list(test_y))

preprocessing test...
language: en
test sequence lengths:
	mean : 311
	95percentile : 722
	99percentile : 1521


In [17]:
model = t.get_classifier()

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

In [18]:
learner = ktrain.get_learner(model, train_data=train, val_data=test, batch_size=6)

In [19]:
learner.fit_onecycle(5e-5, 3, verbose=1)



begin training using onecycle policy with max lr of 5e-05...
Epoch 1/3
19825/19825 [==============================] - 7866s 395ms/step - loss: 0.4677 - accuracy: 0.7940 - val_loss: 0.2442 - val_accuracy: 0.9034
Epoch 2/3
19825/19825 [==============================] - 7844s 395ms/step - loss: 0.2436 - accuracy: 0.9070 - val_loss: 0.2054 - val_accuracy: 0.9216
Epoch 3/3
16498/19825 [=======================>......] - ETA: 20:18 - loss: 0.1530 - accuracy: 0.9432

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [20]:
learner.save_model('news_sources_ktrain_model_2')

In [21]:
predictor = ktrain.get_predictor(learner.model, t)

In [22]:
predictor.save('predictor_news_sources_ktrain_model')

In [25]:
predictor.

['center', 'left', 'right']

In [26]:
learner.validate(class_names=['left', 'center', 'right'])

              precision    recall  f1-score   support

        left       0.96      0.93      0.95      8690
      center       0.93      0.95      0.94     13332
       right       0.91      0.91      0.91      7715

    accuracy                           0.93     29737
   macro avg       0.93      0.93      0.93     29737
weighted avg       0.93      0.93      0.93     29737



array([[ 8111,   408,   171],
       [  204, 12618,   510],
       [   97,   581,  7037]])

In [28]:
doc = text_label['text'].iloc[0]

In [29]:
predictor.predict(doc)

'left'